# Study on Decentraland 

### Retrieve data from decentraland

In [2]:
import pandas as pd
import numpy as np
import datetime
import time
from tqdm import tqdm
from time import sleep

from scipy.interpolate import griddata

import matplotlib.pyplot as plt
import seaborn as sns

import requests
from pandas import json_normalize

pd.options.display.max_columns = None

In [3]:
    # data from decentraland
url_decentraland = "https://api.decentraland.org/"
response_tiles = requests.get(url_decentraland + "v2/tiles")
tiles = pd.DataFrame.from_dict(response_tiles.json()["data"], orient="index").reset_index(drop=True)
tiles = tiles.rename(columns={"id":"xy"})[["xy","x","y","type","estateId","owner","tokenId","name","price"]]

In [4]:
    # df save to csv
today = "220126"
tiles.to_csv('Y:\decentraland_data_retrieve/retrieved_data/tiles/tile_info_{date}.csv'.format(date=today), header=True)

In [5]:
print(len(tiles))
print(tiles['x'].max(), tiles['y'].max())
print(tiles['x'].min(), tiles['y'].min())

92598
163 158
-150 -150


In [6]:
tiles.head(3)

,xy,x,y,type,estateId,owner,tokenId,name,price
0,"-150,150",-150,150,district,1186,0xa65be351527ebcf8c1707d1e444dac38b41a5faf,1157920892373161954235709850086879078022276296...,NaN,NaN
1,"-150,149",-150,149,district,1186,0xa65be351527ebcf8c1707d1e444dac38b41a5faf,1157920892373161954235709850086879078022276296...,NaN,NaN
2,"-150,148",-150,148,district,1186,0xa65be351527ebcf8c1707d1e444dac38b41a5faf,1157920892373161954235709850086879078022276296...,NaN,NaN


#### Private parcel owner

In [7]:
owner_count = pd.DataFrame(tiles[tiles["type"]=='owned']['owner'].value_counts().rename_axis('owner_address').reset_index(name='counts'))

In [8]:
len(owner_count)

6127

In [9]:
owner_count.head(3)

,owner_address,counts
0,0x1463b7162103247c5d464f104f7c9da61dea1bfc,3548
1,0xb096e691b197e4622bc0033a2efda76082b33783,1527
2,0x5326a48a8badfbbf2fb6c4a03ae6685919e3040f,1397


In [ ]:
plt.figure(figsize= (30,10))
plt.boxplot(owner_count['counts'], vert=0)
plt.xscale('log')
plt.show()

## Etherscan

#### Common Things

In [11]:
module = "account"
address = "0x82fbcf5e14abeeefc6c9596b777190d1b5459898"
API_KEY_ETH = "UMSMKQKRPN8PGYIR1Q9HGVE561S393JI5V"

##### Normal Transaction

In [31]:
norm_tx_action = "txlist"
url_norm_tx = 'https://api.etherscan.io/api?module={mdl}&action={act}&address={adrs}&startblock=0&endblock=99999999&page=1&offset=100&sort=asc&apikey={api}'.format(mdl=module, act = norm_tx_action, adrs = address, api=API_KEY_ETH)
response_norm_tx = requests.get(url_norm_tx)

In [ ]:
df_norm_tx = pd.DataFrame.from_dict(response_norm_tx.json()["result"], orient="columns")
df_norm_tx_col = df_norm_tx[["timeStamp","hash","transactionIndex","from","to","value","gas","gasPrice"]]
df_norm_tx_col.head(2)

#### ERC 20 Transaction

In [31]:
# tx_20_action = "tokentx"
# url_20_tx = 'https://api.etherscan.io/api?module={mdl}&action={act}&address={adrs}&page=1&offset=100&startblock=0&endblock=999999999&sort=asc&apikey={api}'.format(mdl=module, act = tx_20_action, adrs = address, api=API_KEY_ETH)
# response_20_tx = requests.get(url_20_tx)

In [ ]:
# df_20_tx = pd.DataFrame.from_dict(response_20_tx.json()["result"], orient="columns")
# df_20_tx_col = df_20_tx[["timeStamp","hash","from","contractAddress","to","value","tokenName","tokenSymbol","tokenDecimal","transactionIndex","gas","gasPrice"]]
# df_20_tx_col['value'] = df_20_tx_col['value'].astype('float').apply(lambda x: round(x*(pow(10,-18)),5))

In [ ]:
# for retrieve ERC20 txn data
for owner in tqdm(owner_count["owner_address"]):
   address = owner

   tx_20_action = "tokentx"
   url_20_tx = 'https://api.etherscan.io/api?module={mdl}&action={act}&address={adrs}&page=1&offset=100&startblock=0&endblock=999999999&sort=asc&apikey={api}'.format(mdl=module, act=tx_20_action, adrs=address, api=API_KEY_ETH)
   response_20_tx = requests.get(url_20_tx)
   try:
       df_20_tx = json_normalize(response_20_tx.json()["result"])
       df_20_tx_col = df_20_tx[["timeStamp", "hash", "from", "contractAddress", "to", "value", "tokenName", "tokenSymbol", "tokenDecimal", "transactionIndex", "gas", "gasPrice"]]
       df_20_tx_col['value'] = df_20_tx_col['value'].astype('float').apply(lambda x: round(x*(pow(10,-18)),5))
   except:
       pass
   df_20_tx_col.to_csv("Y:\decentraland_data_retrieve/retrieved_data/tx_erc20/erc20_{owner}.csv".format(owner=owner), header=True)

print("ERC20 data retrieve done!")

#### ERC 721 Transaction

In [29]:
# tx_721_action = "tokennfttx"
# url_721_tx = 'https://api.etherscan.io/api?module={mdl}&action={act}&address={adrs}&page=1&offset=100&startblock=0&endblock=999999999&sort=asc&apikey={api}'.format(mdl=module, act = tx_721_action, adrs = address, api=API_KEY_ETH)
# response_721_tx = requests.get(url_721_tx)

In [36]:
# df_721_tx = pd.DataFrame.from_dict(response_721_tx.json()["result"], orient="columns")
# df_721_tx_col = df_721_tx[["timeStamp","hash","from","contractAddress","to","tokenID","tokenName","tokenSymbol","tokenDecimal","transactionIndex","gas","gasPrice"]]

In [49]:
# for retrieve ERC721 txn data
for owner in tqdm(owner_count["owner_address"]):
   address = owner

   tx_721_action = "tokennfttx"
   url_721_tx = 'https://api.etherscan.io/api?module={mdl}&action={act}&address={adrs}&page=1&offset=100&startblock=0&endblock=999999999&sort=asc&apikey={api}'.format(mdl=module, act = tx_721_action, adrs = address, api=API_KEY_ETH)
   response_721_tx = requests.get(url_721_tx)

   df_721_tx = pd.DataFrame.from_dict(response_721_tx.json()["result"], orient="columns")
   
   try:
        df_721_tx_col = df_721_tx[["timeStamp","hash","from","contractAddress","to","tokenID","tokenName","tokenSymbol","tokenDecimal","transactionIndex","gas","gasPrice"]]
   except:
       pass
   df_721_tx_col.to_csv("Y:\decentraland_data_retrieve/retrieved_data/tx_erc721/erc721_{owner}.csv".format(owner=owner), header=True)

print("ERC721 data retrieve done!")

100%|██████████| 6090/6090 [1:51:19<00:00,  1.10s/it]  

ERC721 data retrieve done!


Qs
- 몇 명의 유저가 액티브, 랜드를 소유하는지, 